<a href="https://colab.research.google.com/github/bingronglee/SillyCode/blob/main/%20Qwen-Image-Edit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ComfyUI + Qwen-Image-Edit (Google Drive 版)
此 Notebook 使用 Google Drive 儲存模型，避免每次都重新下載

## 特點:
- ✅ 模型儲存在 Google Drive，永久保存
- ✅ 使用 Localtunnel 提供外部訪問
- ✅ 支援 GGUF 量化模型節省顯存

## 📁 步驟 1: 掛載 Google Drive 與環境檢查

In [1]:
from google.colab import drive
import os
import torch

# 掛載雲端硬碟
print("📁 正在掛載 Google Drive...")
drive.mount('/content/drive')

# 定義 Drive 上的模型存放路徑
DRIVE_PATH = "/content/drive/MyDrive/ComfyUI_Models"
os.makedirs(f"{DRIVE_PATH}/unet", exist_ok=True)
os.makedirs(f"{DRIVE_PATH}/clip", exist_ok=True)
os.makedirs(f"{DRIVE_PATH}/vae", exist_ok=True)
os.makedirs(f"{DRIVE_PATH}/loras", exist_ok=True)
print(f"✅ Drive 資料夾已建立: {DRIVE_PATH}")

# 檢查 GPU
if not torch.cuda.is_available():
    print("❌ 警告：未偵測到 GPU，請到「執行階段」→「變更執行階段類型」→ 選擇 GPU")
else:
    print(f"✅ GPU 已準備就緒: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

📁 正在掛載 Google Drive...
Mounted at /content/drive
✅ Drive 資料夾已建立: /content/drive/MyDrive/ComfyUI_Models
✅ GPU 已準備就緒: Tesla T4
   VRAM: 14.7 GB


## 🔧 步驟 2: 安裝基礎工具

In [2]:
# 安裝 aria2 下載工具和 Localtunnel
!apt -q update
!apt -q install -y aria2
!npm install -g localtunnel

print("✅ 基礎工具安裝完成")

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,297 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packa

## 📥 步驟 3: 部署 ComfyUI 與插件

In [3]:
import os

# 克隆 ComfyUI
%cd /content
if not os.path.exists("/content/ComfyUI"):
    print("📥 正在下載 ComfyUI...")
    !git clone https://github.com/comfyanonymous/ComfyUI
else:
    print("✅ ComfyUI 已存在")

%cd /content/ComfyUI

# 固定版本以確保穩定性
!git fetch --all -q
!git reset --hard cd66d72b464fd9d344baa426b50a5f0e5e512f99

# 安裝 PyTorch 和相依套件
print("📦 正在安裝 PyTorch 和相依套件...")
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q -r requirements.txt

# 安裝 GGUF 專用插件
print("🔌 正在安裝 GGUF 插件...")
%cd /content/ComfyUI/custom_nodes
if not os.path.exists("ComfyUI-GGUF"):
    !git clone https://github.com/city96/ComfyUI-GGUF
else:
    print("✅ GGUF 插件已存在")

%cd /content/ComfyUI
print("✅ ComfyUI 部署完成")

/content
📥 正在下載 ComfyUI...
Cloning into 'ComfyUI'...
remote: Enumerating objects: 29316, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 29316 (delta 3), reused 4 (delta 3), pack-reused 29304 (from 2)
Receiving objects: 100% (29316/29316), 75.78 MiB | 24.66 MiB/s, done.
Resolving deltas: 100% (19899/19899), done.
/content/ComfyUI
HEAD is now at cd66d72b convert CLIPTextEncodeSDXL nodes to V3 schema (#9716)
📦 正在安裝 PyTorch 和相依套件...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 116.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 💾 步驟 4: 下載模型到 Google Drive
**注意**: 第一次執行需要下載約 8-10GB 的模型，之後會直接使用 Drive 中的檔案

In [4]:
import os
import subprocess

def smart_download(url, drive_dir, filename):
    """智慧下載：檢查檔案是否已存在於 Drive"""
    local_file = os.path.join(drive_dir, filename)
    if os.path.exists(local_file):
        file_size = os.path.getsize(local_file) / (1024**3)  # 轉換為 GB
        print(f"✅ 模型已存在於 Drive ({file_size:.2f} GB)，跳過下載: {filename}")
    else:
        print(f"🚀 正在下載模型至 Drive: {filename}")
        subprocess.run([
            "aria2c", "--console-log-level=error", "-c",
            "-x", "16", "-s", "16", "-k", "1M",
            url, "-d", drive_dir, "-o", filename
        ], check=True)
        print(f"   ✅ 下載完成: {filename}")

# 定義模型列表
models = [
    (
        "https://huggingface.co/QuantStack/Qwen-Image-Edit-2509-GGUF/resolve/main/Qwen-Image-Edit-2509-Q4_K_M.gguf",
        f"{DRIVE_PATH}/unet",
        "Qwen-Image-Edit-2509-Q4_K_M.gguf"
    ),
    (
        "https://huggingface.co/Comfy-Org/Qwen-Image_ComfyUI/resolve/main/split_files/text_encoders/qwen_2.5_vl_7b_fp8_scaled.safetensors",
        f"{DRIVE_PATH}/clip",
        "qwen_2.5_vl_7b_fp8_scaled.safetensors"
    ),
    (
        "https://huggingface.co/Comfy-Org/Qwen-Image_ComfyUI/resolve/main/split_files/vae/qwen_image_vae.safetensors",
        f"{DRIVE_PATH}/vae",
        "qwen_image_vae.safetensors"
    ),
    (
        "https://huggingface.co/lightx2v/Qwen-Image-Lightning/resolve/main/Qwen-Image-Lightning-4steps-V1.0-bf16.safetensors",
        f"{DRIVE_PATH}/loras",
        "Qwen-Image-Lightning-4steps-V1.0-bf16.safetensors"
    )
]

# 執行下載
print("\n📦 開始檢查並下載模型...\n")
for url, d_path, f_name in models:
    smart_download(url, d_path, f_name)

print("\n✅ 所有模型準備完成")


📦 開始檢查並下載模型...

🚀 正在下載模型至 Drive: Qwen-Image-Edit-2509-Q4_K_M.gguf
   ✅ 下載完成: Qwen-Image-Edit-2509-Q4_K_M.gguf
🚀 正在下載模型至 Drive: qwen_2.5_vl_7b_fp8_scaled.safetensors
   ✅ 下載完成: qwen_2.5_vl_7b_fp8_scaled.safetensors
🚀 正在下載模型至 Drive: qwen_image_vae.safetensors
   ✅ 下載完成: qwen_image_vae.safetensors
🚀 正在下載模型至 Drive: Qwen-Image-Lightning-4steps-V1.0-bf16.safetensors
   ✅ 下載完成: Qwen-Image-Lightning-4steps-V1.0-bf16.safetensors

✅ 所有模型準備完成


## 🔗 步驟 5: 建立軟連結
將 ComfyUI 的模型目錄指向 Google Drive

In [5]:
# 移除舊目錄並建立軟連結
!rm -rf /content/ComfyUI/models/unet
!ln -s {DRIVE_PATH}/unet /content/ComfyUI/models/unet

!rm -rf /content/ComfyUI/models/clip
!ln -s {DRIVE_PATH}/clip /content/ComfyUI/models/clip

!rm -rf /content/ComfyUI/models/vae
!ln -s {DRIVE_PATH}/vae /content/ComfyUI/models/vae

!rm -rf /content/ComfyUI/models/loras
!ln -s {DRIVE_PATH}/loras /content/ComfyUI/models/loras

# 驗證連結
print("✅ 軟連結已建立:")
!ls -lh /content/ComfyUI/models/ | grep '^l'

✅ 軟連結已建立:
lrwxrwxrwx 1 root root   42 Jan 14 08:45 clip -> /content/drive/MyDrive/ComfyUI_Models/clip
lrwxrwxrwx 1 root root   43 Jan 14 08:45 loras -> /content/drive/MyDrive/ComfyUI_Models/loras
lrwxrwxrwx 1 root root   42 Jan 14 08:45 unet -> /content/drive/MyDrive/ComfyUI_Models/unet
lrwxrwxrwx 1 root root   41 Jan 14 08:45 vae -> /content/drive/MyDrive/ComfyUI_Models/vae


## 🚀 步驟 6: 啟動 ComfyUI 服務
使用 Localtunnel 提供外部訪問連結

**重要提示**:
1. 執行後會顯示 Localtunnel 連結和 IP 位址
2. 點擊連結後需要輸入顯示的 IP 位址進行驗證
3. 此 Cell 會持續運行，停止即關閉服務

In [ ]:
import subprocess
import threading
import time
import re

def start_localtunnel():
    """啟動 Localtunnel 隧道"""
    # 等待 ComfyUI 啟動
    time.sleep(10)

    # 獲取本機 IP
    try:
        endpoint_ip = subprocess.check_output(['curl', '-s', 'ifconfig.me']).decode('utf8').strip()
        print(f"\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        print(f"🔑 Localtunnel 驗證密碼: {endpoint_ip}")
        print(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n")
    except:
        print("⚠️ 無法獲取 IP 位址")

    # 啟動隧道
    proc = subprocess.Popen(
        ["npx", "localtunnel", "--port", "8188"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )

    for line in proc.stdout:
        if "your url is:" in line.lower():
            url = line.split("is:")[1].strip()
            print(f"\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
            print(f"🔗 ComfyUI 連結: {url}")
            print(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
            print(f"\n💡 使用說明:")
            print(f"   1. 點擊上方連結")
            print(f"   2. 輸入驗證密碼 (上方顯示的 IP)")
            print(f"   3. 即可開始使用 ComfyUI\n")
            break

# 啟動隧道執行緒
tunnel_thread = threading.Thread(target=start_localtunnel, daemon=True)
tunnel_thread.start()

# 啟動 ComfyUI
print("🚀 正在啟動 ComfyUI 服務...\n")
%cd /content/ComfyUI
!python main.py --listen 0.0.0.0 --port 8188

🚀 正在啟動 ComfyUI 服務...

/content/ComfyUI
Checkpoint files will always be loaded safely.
Total VRAM 15095 MB, total RAM 12976 MB
pytorch version: 2.9.0+cu126
Set vram state to: NORMAL_VRAM
Device: cuda:0 Tesla T4 : cudaMallocAsync

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🔑 Localtunnel 驗證密碼: 34.139.102.183
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🔗 ComfyUI 連結: https://dry-worms-float.loca.lt
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

💡 使用說明:
   1. 點擊上方連結
   2. 輸入驗證密碼 (上方顯示的 IP)
   3. 即可開始使用 ComfyUI

Using pytorch attention
Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
ComfyUI version: 0.3.60
****** User settings have been changed to be stored on the server instead of browser storage. ******
****** For multi-user setups add the --multi-user CLI argument to enable multiple user profiles. ******
ComfyUI frontend version: 1.26.13
[Prompt Server] web root: /usr/local/lib/python3.12/dist-packages/comfyui_frontend_package/static
Traceback (most recent call 

---
## 📝 常見問題

**Q: 為什麼需要 Google Drive?**  
A: 模型檔案約 8-10GB，儲存在 Drive 可避免每次重新下載，節省時間。

**Q: Localtunnel 連結打不開?**  
A: 請確認已輸入正確的 IP 位址進行驗證。

**Q: 出現 CUDA out of memory 錯誤?**  
A: 請確認使用的是 GPU 執行階段，且選擇了 T4 或更高階的 GPU。

**Q: 如何停止服務?**  
A: 點擊「步驟 6」Cell 左側的停止按鈕即可。